In [1]:
from pyetp.client import connect
from pyetp.config import SETTINGS
from pyetp.types import *
import xtgeo

SETTINGS.dataspace = 'demo/pss-data-gateway'
SETTINGS.etp_url = 'ws://localhost:9100'
SETTINGS.application_name ='etpTest'
SETTINGS.application_version='0.0.1'

token = ""

import resqpy.model as rq
import resqpy.property as rqp
import resqpy.unstructured as rug


model = rq.Model('../data/model_hexa_0.epc')
assert model is not None

hexa_uuids = model.uuids(obj_type='UnstructuredGridRepresentation')
hexa = rug.HexaGrid(model, uuid=hexa_uuids[0])
assert hexa is not None
assert hexa.nodes_per_face is not None, "hexamesh object is incomplete"
assert hexa.nodes_per_face_cl is not None, "hexamesh object is incomplete"
assert hexa.faces_per_cell is not None, "hexamesh object is incomplete"
assert hexa.faces_per_cell_cl is not None, "hexamesh object is incomplete"
assert hexa.cell_face_is_right_handed is not None, "hexamesh object is incomplete"

uuids = model.uuids(obj_type='ContinuousProperty')
prop_titles = list(set([rqp.Property(model, uuid=u).title for u in uuids]))
uuids = model.uuids(obj_type='DiscreteProperty')
prop_titles = list(set(prop_titles + [rqp.Property(model, uuid=u).title for u in uuids]))

#
# The optional "points" (dynamic nodes) property is neither ContinuousProperty nor DiscreteProperty: special treatment
#
node_uuids = model.uuids(title = 'points')         
special_prop_titles = list(set([rqp.Property(model, uuid=u).title for u in node_uuids]))
prop_titles = prop_titles + special_prop_titles
async with connect() as client:
    rddms_uris, prop_uris = await client.put_epc_mesh('../data/model_hexa_0.epc', hexa.title, prop_titles, 23031, SETTINGS.duri)

DEBUG:pyetp.client:sending RequestSession MessageHeader(protocol=0, message_type=1, correlation_id=0, message_id=2, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:starting recv loop
DEBUG:pyetp.client:recv OpenSession MessageHeader(protocol=0, message_type=2, correlation_id=2, message_id=1, message_flags=2)
DEBUG:pyetp.client:sending PutDataObjects MessageHeader(protocol=4, message_type=2, correlation_id=0, message_id=4, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv PutDataObjectsResponse MessageHeader(protocol=4, message_type=9, correlation_id=4, message_id=3, message_flags=2)
DEBUG:pyetp.client:sending PutDataArrays MessageHeader(protocol=9, message_type=4, correlation_id=0, message_id=6, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv PutDataArraysResponse MessageHeader(protocol=9, message_type=10, correlation_id=6, message_id=5, message_flags=2)


gts:  GeologicTimeSeries(uuid=018f81ac-562c-11ef-b79c-000d3abe367a, title=None)
put_epc_mesh property_titles ['Temperature', 'Porosity_initial', 'Porosity_decay', 'insulance_thermal', 'Age', 'Radiogenic_heat_production', 'Density_solid', 'LayerID']


DEBUG:pyetp.client:sending PutDataArrays MessageHeader(protocol=9, message_type=4, correlation_id=0, message_id=8, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv PutDataArraysResponse MessageHeader(protocol=9, message_type=10, correlation_id=8, message_id=7, message_flags=2)
DEBUG:pyetp.client:sending PutDataArrays MessageHeader(protocol=9, message_type=4, correlation_id=0, message_id=10, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv PutDataArraysResponse MessageHeader(protocol=9, message_type=10, correlation_id=10, message_id=9, message_flags=2)
DEBUG:pyetp.client:sending PutDataArrays MessageHeader(protocol=9, message_type=4, correlation_id=0, message_id=12, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv PutDataArraysResponse MessageHeader(protocol=9, message_type=10, correlation_id=12, message_id=11, message_flags=2)
DEBUG:pyetp.client:sending PutDataArrays MessageHeader(protocol=9, message_type=4, correlation_id=0, messag

In [2]:
uns_uri = rddms_uris[2]
epc = rddms_uris[0]
temperature_uri = prop_uris["Temperature"][1][0]

In [3]:
x = 3000
y = 6000
top = 0
bottom = 1000
async with connect() as client:
    v = await client.get_epc_mesh_property_x_y(epc,uns_uri,temperature_uri,x,y,top, bottom)

DEBUG:pyetp.client:sending RequestSession MessageHeader(protocol=0, message_type=1, correlation_id=0, message_id=2, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:starting recv loop
DEBUG:pyetp.client:recv OpenSession MessageHeader(protocol=0, message_type=2, correlation_id=2, message_id=1, message_flags=2)
DEBUG:pyetp.client:sending GetDataObjects MessageHeader(protocol=4, message_type=1, correlation_id=0, message_id=4, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv GetDataObjectsResponse MessageHeader(protocol=4, message_type=4, correlation_id=4, message_id=3, message_flags=2)
DEBUG:pyetp.client:sending GetDataArrayMetadata MessageHeader(protocol=9, message_type=6, correlation_id=0, message_id=6, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv GetDataArrayMetadataResponse MessageHeader(protocol=9, message_type=7, correlation_id=6, message_id=5, message_flags=2)
DEBUG:pyetp.client:sending GetDataArrays MessageHeader(protocol=9, mes

In [4]:
v

{'depth': array([ 200.,  300.,  400.,  500.,  600.,  700.,  800.,  900., 1000.]),
 'values': array([ 5.        ,  9.13037713, 14.03349246, 18.93660779, 23.82594922,
        28.71433603, 33.60272284, 38.46250594, 42.41760754])}